In [45]:
import argparse
import os
import time
import math
import numpy as np
import random
import sys
import json

import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

from utils import to_gpu, Corpus, batchify
from models import Seq2Seq2Decoder, Seq2Seq, MLP_D, MLP_G, MLP_Classify
import shutil

In [46]:
# DATA_PATH = "./data"
OUTF = "yelp_example"
MODELF = "model_output"
VOCAB_SIZE = 8000
MAXLEN = 30
LOWERCASE = True
EMSIZE = 300
NHIDDEN = 512
BATCH_SIZE = 64
NLAYERS = 1
NOISE_R = 0.1
DROPOUT = 0.0
HIDDEN_INIT = True ## ? Is this the correct default value?
Z_SIZE = 32
#Changed the architecture: Removed batch norm layer and made it dense
#Issue: Because we're handling data for +ve and -ve classes separately, the batch norm is skewed towards the latter dataset
#While evaluating the former dataset, these metrics are wrong
#Alt Solution: Run a few forward passes without gradients enabled in model.train() mode, so the batch norm is recitified
ARCH_CLASSIFY = '128-128-128' 
ARCH_D = '128-128'
ARCH_G = '128-128'
# learning rates
LR_AE = 1
LR_GAN_G = 1e-04
LR_GAN_D = 1e-04
LR_CLASSIFY = 1e-04
BETA1 = 0.4
CUDA = True
SEED = 1111
LAMBDA_CLASS = 1
CLIP = 1
TEMP = 1
LOG_INTERVAL = 200
NITERS_GAN_SCHEDULE = ''
GRAD_LAMBDA = 0.01
GAN_GP_LAMBDA = 0.1
EPOCHS = 0
NITERS_AE = 1
NITERS_GAN_G = 1
NITERS_GAN_D = 5
NITERS_GAN_AE = 1
NOISE_ANNEAL = 0.9995
DATA_PATH = './data/'

In [47]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# make output directory if it doesn't already exist
if os.path.isdir(OUTF):
    shutil.rmtree(OUTF)
os.makedirs(OUTF)

# Set the random seed manually for reproducibility.
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    if not CUDA:
        print("WARNING: You have a CUDA device, "
              "so you should probably run with --cuda")
    else:
        torch.cuda.manual_seed(SEED)

In [48]:
###############################################################################
# Load data
###############################################################################

label_ids = {"pos": 1, "neg": 0}
id2label = {1:"pos", 0:"neg"}

# (Path to textfile, Name, Use4Vocab)
datafiles = [(os.path.join(DATA_PATH, "valid1.txt"), "valid1", False),
             (os.path.join(DATA_PATH, "valid2.txt"), "valid2", False),
             (os.path.join(DATA_PATH, "test1.txt"), "test1", False),
             (os.path.join(DATA_PATH, "test0.txt"), "test2", False),
             (os.path.join(DATA_PATH, "train1.txt"), "train1", True),
             (os.path.join(DATA_PATH, "train2.txt"), "train2", True)]
vocabdict = None
corpus = Corpus(datafiles,
                maxlen=MAXLEN,
                vocab_size=VOCAB_SIZE,
                lowercase=LOWERCASE,
                vocab=vocabdict)

# dumping vocabulary
with open('{}/vocab.json'.format(OUTF), 'w') as f:
    json.dump(corpus.dictionary.word2idx, f)

# save arguments
ntokens = len(corpus.dictionary.word2idx)
print("Vocabulary Size: {}".format(ntokens))

eval_batch_size = 100
val_data = batchify(corpus.data['valid1'], eval_batch_size, shuffle=False)
val2_data = batchify(corpus.data['valid2'], eval_batch_size, shuffle=False)
test1_data = batchify(corpus.data['test1'], eval_batch_size, shuffle=False)
test2_data = batchify(corpus.data['test2'], eval_batch_size, shuffle=False)
train1_data = batchify(corpus.data['train1'], BATCH_SIZE, shuffle=True)
train2_data = batchify(corpus.data['train2'], BATCH_SIZE, shuffle=True)

print("Loaded data!")

Original vocab 8092; Pruned to 8004
Number of sentences dropped from ./data/valid1.txt: 0 out of 38205 total
Number of sentences dropped from ./data/valid2.txt: 3 out of 25278 total
Number of sentences dropped from ./data/test1.txt: 0 out of 76392 total
Number of sentences dropped from ./data/test0.txt: 5 out of 50278 total
Number of sentences dropped from ./data/train1.txt: 1 out of 267314 total
Number of sentences dropped from ./data/train2.txt: 4 out of 176787 total
Vocabulary Size: 8004
382 batches
252 batches
763 batches
502 batches
4176 batches
2762 batches
Loaded data!


In [49]:
###############################################################################
# Build the models
###############################################################################

ntokens = len(corpus.dictionary.word2idx)
autoencoder = Seq2Seq2Decoder(emsize=EMSIZE,
                      nhidden=NHIDDEN,
                      ntokens=ntokens,
                      nlayers=NLAYERS,
                      noise_r=NOISE_R,
                      hidden_init=HIDDEN_INIT,
                      dropout=DROPOUT,
                      gpu=CUDA)

gan_gen = MLP_G(ninput=Z_SIZE, noutput=NHIDDEN, layers=ARCH_G)
gan_disc = MLP_D(ninput=NHIDDEN, noutput=1, layers=ARCH_D)
classifier = MLP_Classify(ninput=NHIDDEN, noutput=1, layers=ARCH_CLASSIFY)
g_factor = None

print(autoencoder)
print(gan_gen)
print(gan_disc)
print(classifier)

optimizer_ae = optim.SGD(autoencoder.parameters(), lr=LR_AE)
optimizer_gan_g = optim.Adam(gan_gen.parameters(),
                             lr=LR_GAN_G,
                             betas=(BETA1, 0.999))
optimizer_gan_d = optim.Adam(gan_disc.parameters(),
                             lr=LR_GAN_D,
                             betas=(BETA1, 0.999))
#### classify
optimizer_classify = optim.Adam(classifier.parameters(),
                                lr=LR_CLASSIFY,
                                betas=(BETA1, 0.999))

criterion_ce = nn.CrossEntropyLoss()

if CUDA:
    autoencoder = autoencoder.cuda()
    gan_gen = gan_gen.cuda()
    gan_disc = gan_disc.cuda()
    classifier = classifier.cuda()
    criterion_ce = criterion_ce.cuda()

Seq2Seq2Decoder(
  (embedding): Embedding(8004, 300)
  (embedding_decoder1): Embedding(8004, 300)
  (embedding_decoder2): Embedding(8004, 300)
  (encoder): LSTM(300, 512, batch_first=True)
  (decoder1): LSTM(812, 512, batch_first=True)
  (decoder2): LSTM(812, 512, batch_first=True)
  (linear): Linear(in_features=512, out_features=8004, bias=True)
)
MLP_G(
  (layer1): Linear(in_features=32, out_features=128, bias=True)
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activation1): ReLU()
  (layer2): Linear(in_features=128, out_features=128, bias=True)
  (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activation2): ReLU()
  (layer7): Linear(in_features=128, out_features=512, bias=True)
)
MLP_D(
  (layer1): Linear(in_features=512, out_features=128, bias=True)
  (activation1): LeakyReLU(negative_slope=0.2)
  (layer2): Linear(in_features=128, out_features=128, bias=True)
  (bn2): BatchNorm1d(128, eps=1e-0

In [53]:
###############################################################################
# Training code
###############################################################################
               
def load_model():
    print("Loading models from {}".format(MODELF))
    if os.path.exists('{}/autoencoder_model.pt'.format(MODELF)):
        autoencoder.load_state_dict(torch.load('{}/autoencoder_model.pt'.format(MODELF)))
    if os.path.exists('{}/gan_gen_model.pt'.format(MODELF)):    
        gan_gen.load_state_dict(torch.load('{}/gan_gen_model.pt'.format(MODELF)))
    if os.path.exists('{}/gan_disc_model.pt'.format(MODELF)):    
        gan_disc.load_state_dict(torch.load('{}/gan_disc_model.pt'.format(MODELF)))
    if os.path.exists('{}/classifier_model.pt'.format(MODELF)):            
        classifier.load_state_dict(torch.load('{}/classifier_model.pt'.format(MODELF)))

def fgsm_attack(sentence_embedding, epsilon, data_grad):
    # Collect the element-wise sign of the data gradient
    sign_data_grad = data_grad.sign()
    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_embedding = sentence_embedding + epsilon*sign_data_grad
    # Adding clipping to maintain [0,1] range
    #is this required?
    #perturbed_image = torch.clamp(perturbed_image, 0, 1)
    # Return the perturbed image
    return perturbed_embedding

def pgd_attack():
    pass

def evaluate_example(whichclass, data):
    pass

def evaluate_classifier(whichclass, data_source, epoch=0, perturb=False, epsilon=0.0):
    classifier.eval()
    autoencoder.eval()
    total_loss = 0
    all_accuracies = 0
    bcnt = 0
    #commenting as we need gradients
    #with torch.no_grad():
    for i, batch in enumerate(data_source):
        source, target, lengths = batch
        source = to_gpu(CUDA, Variable(source))
        labels = to_gpu(CUDA, Variable(torch.zeros(source.size(0)).fill_(whichclass-1)))
        
        code = autoencoder(0, source, lengths, noise=False, encode_only=True).detach()
        code.requires_grad = True
        scores = classifier(code)
        classify_loss = F.binary_cross_entropy(scores.squeeze(1), labels)
        classifier.zero_grad()
        classify_loss.backward()
        
        code_grad = code.grad.data
        perturbed_code = fgsm_attack(code, epsilon, code_grad)
        scores = classifier(perturbed_code)
        classify_loss = F.binary_cross_entropy(scores.squeeze(1), labels)
        classify_loss = classify_loss.cpu().item()

        pred = scores.data.round().squeeze(1)
        accuracy = pred.eq(labels.data).float().mean()

        total_loss += classify_loss
        all_accuracies += accuracy
        bcnt += 1
    return total_loss/len(data_source), all_accuracies/bcnt

#loading pre-trained weights

load_model()

#without attack
print('Evaluating classifier on validation data')
test_loss_classifier, test_accuracy_classifier = evaluate_classifier(1, test1_data)
print('Classifier on Data 1, Loss:{} Accuracy:{}'.format(test_loss_classifier, test_accuracy_classifier))

test_loss_classifier, test_accuracy_classifier = evaluate_classifier(2, test2_data)
print('Classifier on Data 2, Loss:{} Accuracy:{}'.format(test_loss_classifier, test_accuracy_classifier))

#fgsm 
epsilon = [0.000, 0.0005, 0.001, 0.0015, 0.002, 0.0025, 0.003]
for e in epsilon:
    print('Epsilon: {}'.format(e))
    test_loss_classifier, test_accuracy_classifier = evaluate_classifier(1, test1_data, 0, True, e)
    print('Classifier on Data 1, Loss:{} Accuracy:{}'.format(test_loss_classifier, test_accuracy_classifier))
    
    test_loss_classifier, test_accuracy_classifier = evaluate_classifier(2, test2_data, 0, True, e)
    print('Classifier on Data 2, Loss:{} Accuracy:{}'.format(test_loss_classifier, test_accuracy_classifier))

Loading models from model_output
Evaluating classifier on validation data
Classifier on Data 1, Loss:0.17387842319583519 Accuracy:0.9248232841491699
Classifier on Data 2, Loss:0.26193418606106506 Accuracy:0.8982877135276794
Epsilon: 0.0
Classifier on Data 1, Loss:0.17387842319583519 Accuracy:0.9248232841491699
Classifier on Data 2, Loss:0.26193418606106506 Accuracy:0.8982877135276794
Epsilon: 0.0005
Classifier on Data 1, Loss:0.4073128544799935 Accuracy:0.8168018460273743
Classifier on Data 2, Loss:0.6338521083631363 Accuracy:0.7281673550605774
Epsilon: 0.001
Classifier on Data 1, Loss:0.8295734343881844 Accuracy:0.64922696352005
Classifier on Data 2, Loss:1.3197042420803313 Accuracy:0.4825897216796875
Epsilon: 0.0015
Classifier on Data 1, Loss:1.4723540454673267 Accuracy:0.45005276799201965
Classifier on Data 2, Loss:2.32494501573631 Accuracy:0.25681260228157043
Epsilon: 0.002
Classifier on Data 1, Loss:2.295497343393328 Accuracy:0.2662908732891083
Classifier on Data 2, Loss:3.5174518